INF6804<br>
Polytechnique Montréal<br>
TP3: Détection et suivi de multiples objets d'intérêt<br>

Auteurs:<br>
Marc-Olivier Bélanger - 1840713<br>
Pierre-Luc Chartier - 1805679

### Déclarations globales

In [89]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.optimize as sci
from skimage import io
import torchvision.models as models
import torchvision.transforms as transforms
import urllib.request as request

In [90]:
COCO_NAMES = ['background']
label_names = request.urlopen('https://raw.githubusercontent.com/gabilodeau/INF6804/master/utils/coco-labels.txt')
for label_name in label_names.readlines():
    COCO_NAMES.append(label_name.strip().decode('UTF-8'))

IOU_THRESHOLD = 0.5
SCORE_THRESHOLD = 0.95

model = models.detection.maskrcnn_resnet50_fpn(pretrained=True).eval()

### Détection des objets (Mask R-CNN)

In [91]:
def detect_objects(path_to_img):
    img = io.imread(path_to_img)
    preprocess = transforms.Compose([transforms.ToTensor()])
    tensors = [preprocess(img)]
    
    predictions = model(tensors)
    boxes = []
    
    for p in predictions:
        for j, score in enumerate(p['scores']):
            if score >= SCORE_THRESHOLD:
                label = p['labels'][j]
                is_cup_or_person = (COCO_NAMES[label] == "cup") or (COCO_NAMES[label] == "person")

                if is_cup_or_person:
                    boxes.append(p['boxes'][j])
    return boxes

### Description des objets (Histogramme de couleurs)

### Association des données (Algorithme hongrois)

### Algorithme principal

In [94]:
# keep a set of tracked objects 
# update it as needed e.g. clear the objects undetected since the last n seconds (n*fps frames)
tracked_objects = {}

# pour chaque image:
#  call détection d'objets
boxes = detect_objects("~/Downloads/TP3_data/frames/frame1.jpg")
print(len(boxes))

#  call description d'objets pour construire la matrice des coûts d'association
#  call algo hongrois et associer les objets

#  pour les objets détectés non-associés à cette étape (nouveaux objets ou réapparition)
#   update tracked objects (add to set (create their id, set their count to 0) or reset count)

#  pour les objets non-détectés à cette itération (in tracked, but not in current_detected)
#   update tracked objects (increase count, if count == threshold, remove them)

#  write bboxe coordinates of each detected and associated object (i.e. objects in tracked objects with count = 0)

3


### MOT17-11: Forward moving camera in a busy shopping mall

In [ ]:
# calcul du MOTA en utilisant IoU > 50%

### MOT17-05: Street scene from a moving platform

In [ ]:
# calcul du MOTA en utilisant IoU > 50%

### MOT17-13 : Filmed from a bus on a busy intersection

In [2]:
# calcul du MOTA en utilisant IoU > 50%

### Résultats sur la trame fournie

In [3]:
# call algo principal avec les trames des tasses